In [7]:
import pandas as pd

In [8]:
# Load data
apl_df = pd.read_json('ready-data/nodes-apr.json')
bts_df = pd.read_json('ready-data/nodes-bts.json')
mrt_df = pd.read_json('ready-data/nodes-mrt.json')
taxi_df = pd.read_json('ready-data/nodes-taxi.json')
inter_df = pd.read_json('ready-data/nodes-inter.json')
geo_dist_df = pd.read_json('ready-data/nodes-geo_dist.json')
inter_list = list(inter_df.set_index('origins').index)

In [9]:
duration_avg = apl_df['duration'].append(bts_df['duration'].append(mrt_df['duration'].append(taxi_df['duration']))).mean()
cost_avg = apl_df['cost'].append(bts_df['cost'].append(mrt_df['cost'].append(taxi_df['cost']))).mean()
ratio = duration_avg/cost_avg
print(ratio)

12.484245959663399


In [10]:
def gen_successors(node):
    global last_index
    state = node[0]
    node_id = node[1]
    parent_index = node[2]
    level = node[3]
    price = node[4]
    duration = node[5]
    fn = node[6]
    goal_system = goal_state[:3]
    ret = []
    
    #Go taxi
    look = taxi_df
    next_dest = list(look.set_index('origins').loc[state]['destination'])
    for dest in next_dest:
        if goal_state not in look.set_index('origins').loc[dest]['destination']:
            continue
                #(state, node_id, parent_id, level, price, time, f, system)
        ret += [(dest, last_index+1, node_id, level+1, int(price+cal_price(state, dest, look)), int(duration+cal_time(state, dest, look)), int(sum_cost(price+cal_price(state, dest, look), duration+cal_time(state, dest, look))+g(dest, goal_state, taxi_df)), 'TAX')]
        last_index += 1
        
    #check system
    if node[7] == 'TAX':
        system = state[:3]
    else:
        system = node[7]

    #Go direct
    if system == goal_system:
        look = look_at(system)
        next_dest = list(look.set_index('origins').loc[state]['destination'])
        for dest in next_dest:
                    #(state, node_id, parent_id, level, price, time, f, system)
            ret += [(dest, last_index+1, node_id, level+1, int(price+cal_price(state, dest, look)), int(duration+cal_time(state, dest, look)), int(sum_cost(price+cal_price(state, dest, look), duration+cal_time(state, dest, look))+g(dest, goal_state, look)), system)]
            last_index += 1
            
    #Go interchange
    if system != goal_system:
#         print(system)
#         print(goal_system)
        look = inter_df
        inter = {} #{from : to}
#         next_dest = []
        for station in look['origins']:
            if station[:3] == system:
                inter[station] = ''
        for station in inter.keys():
            if look.set_index('origins').loc[station]['destination'][:3] == goal_system:
                inter[station] = look.set_index('origins').loc[station]['destination']
#                 next_dest += [look.set_index('origins').loc[station]['destination']]
        for dest in inter.items(): 
            if dest[1] != '':
                print(dest)
                ret += [(dest[1], last_index+1, node_id, level+1, int(price+cal_price(state, dest[0], look_at(system))), int(duration+cal_time(state, dest[0], look_at(system))), int(sum_cost(price+cal_price(state, dest[0], look_at(system)), duration+cal_time(state, dest[0], look_at(system)))+g(dest[1], goal_state, look_at(dest[1][:3]) )), dest[1][:3])]
                last_index += 1
            
    return ret

def sum_cost(price, time):
    return (time * time_ratio) + ((12.5 * price) * price_ratio)

def look_at(system):
    if system == 'MRT':
        look = mrt_df
    elif system == 'BTS':
        look = bts_df
    elif system == 'AIR':
        look = apl_df
    elif system == 'TAX':
        look = taxi_df
    return look

def g(state, dest, look):
    global time_ratio
    global price_ratio
    total_cost = 0
    price = cal_price(state, dest, look)
    time = cal_time(state, dest, look)
    total_cost += (time * time_ratio) + ((12.5 * price) * price_ratio)
    return total_cost

def cal_price(state, dest, look):
    if state == dest:
        return 0
    price = look.set_index('origins').loc[state].set_index('destination').loc[dest][2]
    return price

def cal_time(state, dest, look):
    if state == dest:
        return 0
    time = look.set_index('origins').loc[state].set_index('destination').loc[dest][1]
    return time

# def h(state):#displacement
#     if state == goal_state:
#         return 0
#     return geo_dist_df.set_index('origins').loc[state].set_index('destination').loc[goal_state][0]

def is_goal(node):
    return node[0] == goal_state

def visited(child):
    global visited_dest
    visited_dest = set()
    for i in visited_node.values():
        visited_dest.add(i[0])
    return child[0] in visited_dest
    

def insert_all(node, fringe):
    global total_nodes
    children = gen_successors(node)
    total_nodes += len(children)
    for child in children:
        inserted = False
        for j in range(len(fringe)):
            if child[6] < fringe[j][6]:
                if not visited(child):
                    fringe[j:j] = [child]
                inserted = True
                break
        if not inserted:
            if not visited(child):
                fringe.append(child)
#     print(fringe)
                
def show_result(front):
    global visited_node
    current_node = front
    parent_index = current_node[1]
    print('Founded')
    while True:
        print(current_node)
        parent_index = current_node[2]
        if parent_index == -1:
            break
        current_node = visited_node[parent_index]
                
def a_star(start_node):
    global last_index
    global visited_node
    global visited_dest
    last_index = 0
    fringe = [start_node]
    visited_node = {}
#     visited_dest = set()
    while True:
        if len(fringe) == 0:
            print('Not found')
            break
        front = fringe[0]
        visited_node[front[1]] = front
#         visited_dest.add(front[0])
        fringe = fringe[1:]
        if is_goal(front):
            show_result(front)
            return True
        print(front)
        insert_all(front, fringe)

In [15]:
start_state = 'BTS คูคต'
goal_state = 'MRT สามย่าน'
time_ratio = 0
price_ratio = 1

visited_node = {}
last_index = 0
total_nodes = 0
#(state, node_id, parent_id, level, price, time, f, system)
start_node = (start_state,0,-1,0,0,0,0,start_state[:3]) 
start_node

('BTS คูคต', 0, -1, 0, 0, 0, 0, 'BTS')

In [16]:
a_star(start_node)

('BTS คูคต', 0, -1, 0, 0, 0, 0, 'BTS')
('BTS ห้าแยกลาดพร้าว', 'MRT พหลโยธิน')
('BTS หมอชิต', 'MRT สวนจตุจักร')
('BTS บางหว้า', 'MRT บางหว้า')
('BTS ศาลาแดง', 'MRT สีลม')
('BTS อโศก', 'MRT สุขุมวิท')
('MRT พหลโยธิน', 2, 0, 1, 0, 1560, 525, 'MRT')
('MRT สวนจตุจักร', 3, 0, 1, 0, 1740, 525, 'MRT')
Founded
('MRT สามย่าน', 39, 2, 2, 42, 3960, 525, 'MRT')
('MRT พหลโยธิน', 2, 0, 1, 0, 1560, 525, 'MRT')
('BTS คูคต', 0, -1, 0, 0, 0, 0, 'BTS')


True

In [17]:
#(state, node_id, parent_id, level, price, time, f, system)
visited_node

{0: ('BTS คูคต', 0, -1, 0, 0, 0, 0, 'BTS'),
 2: ('MRT พหลโยธิน', 2, 0, 1, 0, 1560, 525, 'MRT'),
 3: ('MRT สวนจตุจักร', 3, 0, 1, 0, 1740, 525, 'MRT'),
 39: ('MRT สามย่าน', 39, 2, 2, 42, 3960, 525, 'MRT')}

In [19]:
visited_dest

{'BTS คูคต', 'MRT พหลโยธิน', 'MRT สวนจตุจักร'}